import dependencies

In [1]:
import tensorflow as tf
import numpy as np
import random
import time
import sys

RNN with num_layers LSTM layers and a fully-connected output layer
The network allows for a dynamic number of iterations, depending on the inputs it receives.

out (fc layer; out_size) <- lstm <- lstm <- in (in_size)

In [2]:
class ModelNetwork:
    def __init__(self, in_size, lstm_size, num_layers, out_size, session, learning_rate=0.003, name="rnn"):
        self.scope = name

        self.in_size = in_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.out_size = out_size

        self.session = session

        self.learning_rate = tf.constant( learning_rate )

        # Last state of LSTM, used when running the network in TEST mode
        self.lstm_last_state = np.zeros((self.num_layers*2*self.lstm_size,))

        with tf.variable_scope(self.scope):
            ## (batch_size, timesteps, in_size)
            self.xinput = tf.placeholder(tf.float32, shape=(None, None, self.in_size), name="xinput")
            self.lstm_init_value = tf.placeholder(tf.float32, shape=(None, self.num_layers*2*self.lstm_size), name="lstm_init_value")

            # LSTM
            self.lstm_cells = [ tf.contrib.rnn.BasicLSTMCell(self.lstm_size, forget_bias=1.0, state_is_tuple=False) for i in range(self.num_layers)]
            self.lstm = tf.contrib.rnn.MultiRNNCell(self.lstm_cells, state_is_tuple=False)

            # Iteratively compute output of recurrent network
            outputs, self.lstm_new_state = tf.nn.dynamic_rnn(self.lstm, self.xinput, initial_state=self.lstm_init_value, dtype=tf.float32)

            # Linear activation (FC layer on top of the LSTM net)
            self.rnn_out_W = tf.Variable(tf.random_normal( (self.lstm_size, self.out_size), stddev=0.01 ))
            self.rnn_out_B = tf.Variable(tf.random_normal( (self.out_size, ), stddev=0.01 ))

            outputs_reshaped = tf.reshape( outputs, [-1, self.lstm_size] )
            network_output = ( tf.matmul( outputs_reshaped, self.rnn_out_W ) + self.rnn_out_B )

            batch_time_shape = tf.shape(outputs)
            self.final_outputs = tf.reshape( tf.nn.softmax( network_output), (batch_time_shape[0], batch_time_shape[1], self.out_size) )


            ## Training: provide target outputs for supervised training.
            self.y_batch = tf.placeholder(tf.float32, (None, None, self.out_size))
            y_batch_long = tf.reshape(self.y_batch, [-1, self.out_size])

            self.cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=network_output, labels=y_batch_long) )
            self.train_op = tf.train.RMSPropOptimizer(self.learning_rate, 0.9).minimize(self.cost)


    ## Input: X is a single element, not a list!
    def run_step(self, x, init_zero_state=True):
        ## Reset the initial state of the network.
        if init_zero_state:
            init_value = np.zeros((self.num_layers*2*self.lstm_size,))
        else:
            init_value = self.lstm_last_state

        out, next_lstm_state = self.session.run([self.final_outputs, self.lstm_new_state], feed_dict={self.xinput:[x], self.lstm_init_value:[init_value]   } )

        self.lstm_last_state = next_lstm_state[0]

        return out[0][0]


    ## xbatch must be (batch_size, timesteps, input_size)
    ## ybatch must be (batch_size, timesteps, output_size)
    def train_batch(self, xbatch, ybatch):
        init_value = np.zeros((xbatch.shape[0], self.num_layers*2*self.lstm_size))

        cost, _ = self.session.run([self.cost, self.train_op], feed_dict={self.xinput:xbatch, self.y_batch:ybatch, self.lstm_init_value:init_value   } )

        return cost

Embed string to character-arrays -- it generates an array len(data) x len(vocab)

Vocab is a list of elements

In [3]:
def embed_to_vocab(data_, vocab):
    data = np.zeros((len(data_), len(vocab)))

    cnt=0
    for s in data_:
        v = [0.0]*len(vocab)
        v[vocab.index(s)] = 1.0
        data[cnt, :] = v
        cnt += 1

    return data

def decode_embed(array, vocab):
    return vocab[ array.index(1) ]

In [4]:
ckpt_file = ""
TEST_PREFIX = "The " # Prefix to prompt the network in test mode

## Load the data
data_ = ""
with open('data/dostoevski.txt', 'r', encoding="utf-8") as f:
    data_ += f.read()
data_ = data_.lower()

## Convert to 1-hot coding
vocab = list(set(data_))

data = embed_to_vocab(data_, vocab)


in_size = out_size = len(vocab)
lstm_size = 256
num_layers = 2
batch_size = 64 #128
time_steps = 100 #50

NUM_TRAIN_BATCHES = 19500

LEN_TEST_TEXT = 500 # Number of test characters of text to generate after training the network



## Initialize the network
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)

net = ModelNetwork(in_size = in_size,lstm_size = lstm_size,num_layers = num_layers,out_size = out_size,session = sess,learning_rate = 0.003)

sess.run(tf.global_variables_initializer())

saver = tf.train.Saver(tf.global_variables())


train network

In [5]:
if ckpt_file == "":
    last_time = time.time()

    batch = np.zeros((batch_size, time_steps, in_size))
    batch_y = np.zeros((batch_size, time_steps, in_size))

    possible_batch_ids = range(data.shape[0]-time_steps-1)
    for i in range(NUM_TRAIN_BATCHES):
        # Sample time_steps consecutive samples from the dataset text file
        batch_id = random.sample( possible_batch_ids, batch_size )

        for j in range(time_steps):
            ind1 = [k+j for k in batch_id]
            ind2 = [k+j+1 for k in batch_id]

            batch[:, j, :] = data[ind1, :]
            batch_y[:, j, :] = data[ind2, :]


        cst = net.train_batch(batch, batch_y)

        if (i%10) == 0:
            new_time = time.time()
            diff = new_time - last_time
            last_time = new_time

            print("batch: ",i,"   loss: ",cst,"   speed: ",(100.0/diff)," batches / s")

saver.save(sess, "./model.ckpt")

batch:  0    loss:  4.55531    speed:  134.7300470878544  batches / s
batch:  10    loss:  4.55334    speed:  14.81876504756129  batches / s
batch:  20    loss:  4.54968    speed:  15.092368025296343  batches / s
batch:  30    loss:  4.54402    speed:  15.09004296285082  batches / s
batch:  40    loss:  4.53532    speed:  15.092319149260781  batches / s
batch:  50    loss:  4.52302    speed:  14.757310848490205  batches / s
batch:  60    loss:  4.5043    speed:  13.635605271011581  batches / s
batch:  70    loss:  4.47584    speed:  14.512521367135438  batches / s
batch:  80    loss:  4.4039    speed:  14.646799726375427  batches / s
batch:  90    loss:  3.5386    speed:  14.963249939298702  batches / s
batch:  100    loss:  3.34338    speed:  14.254279270369738  batches / s
batch:  110    loss:  3.29796    speed:  14.883958076554281  batches / s
batch:  120    loss:  3.24736    speed:  15.071814289610046  batches / s
batch:  130    loss:  3.23931    speed:  15.094640567379153  batches

batch:  1130    loss:  1.53284    speed:  14.766106025982872  batches / s
batch:  1140    loss:  1.53557    speed:  14.685680244695979  batches / s
batch:  1150    loss:  1.48756    speed:  14.905157439684242  batches / s
batch:  1160    loss:  1.44543    speed:  14.91851892110148  batches / s
batch:  1170    loss:  1.49895    speed:  13.897780913222137  batches / s
batch:  1180    loss:  1.54745    speed:  14.86958556675586  batches / s
batch:  1190    loss:  1.46449    speed:  14.89176964137615  batches / s
batch:  1200    loss:  1.46642    speed:  14.878459705052546  batches / s
batch:  1210    loss:  1.49202    speed:  14.867362362005236  batches / s
batch:  1220    loss:  1.45657    speed:  14.862931113358384  batches / s
batch:  1230    loss:  1.49903    speed:  14.880667224314198  batches / s
batch:  1240    loss:  1.54238    speed:  14.86295692085383  batches / s
batch:  1250    loss:  1.44352    speed:  14.889555655656164  batches / s
batch:  1260    loss:  1.5107    speed:  1

batch:  2250    loss:  1.27174    speed:  14.803459494913913  batches / s
batch:  2260    loss:  1.28686    speed:  14.61467545750828  batches / s
batch:  2270    loss:  1.35364    speed:  15.216770292109613  batches / s
batch:  2280    loss:  1.30509    speed:  15.247022947360179  batches / s
batch:  2290    loss:  1.34347    speed:  15.233036951056611  batches / s
batch:  2300    loss:  1.31424    speed:  15.235385814355713  batches / s
batch:  2310    loss:  1.32746    speed:  15.244672156992022  batches / s
batch:  2320    loss:  1.2964    speed:  15.170494376870991  batches / s
batch:  2330    loss:  1.31666    speed:  15.108429525221215  batches / s
batch:  2340    loss:  1.28756    speed:  15.133655600787659  batches / s
batch:  2350    loss:  1.2654    speed:  15.228381758243762  batches / s
batch:  2360    loss:  1.31874    speed:  15.242361419400718  batches / s
batch:  2370    loss:  1.27517    speed:  15.257510023155612  batches / s
batch:  2380    loss:  1.29712    speed: 

batch:  3370    loss:  1.21979    speed:  14.653283512519684  batches / s
batch:  3380    loss:  1.19229    speed:  14.417048309216604  batches / s
batch:  3390    loss:  1.22541    speed:  14.70189270870872  batches / s
batch:  3400    loss:  1.19338    speed:  13.844943383765237  batches / s
batch:  3410    loss:  1.22944    speed:  13.750074416801754  batches / s
batch:  3420    loss:  1.15342    speed:  13.227303502449002  batches / s
batch:  3430    loss:  1.21378    speed:  11.981989352857209  batches / s
batch:  3440    loss:  1.16012    speed:  10.798362053032283  batches / s
batch:  3450    loss:  1.19597    speed:  12.291586766642325  batches / s
batch:  3460    loss:  1.16301    speed:  12.026892637339653  batches / s
batch:  3470    loss:  1.22725    speed:  13.402662104936516  batches / s
batch:  3480    loss:  1.19416    speed:  13.437066546653805  batches / s
batch:  3490    loss:  1.17664    speed:  13.426947223213093  batches / s
batch:  3500    loss:  1.18071    speed

batch:  4490    loss:  1.11205    speed:  13.913784246816755  batches / s
batch:  4500    loss:  1.10452    speed:  13.683743136194924  batches / s
batch:  4510    loss:  1.122    speed:  13.681871972403897  batches / s
batch:  4520    loss:  1.10597    speed:  13.85194460733409  batches / s
batch:  4530    loss:  1.15605    speed:  13.840411819092596  batches / s
batch:  4540    loss:  1.07068    speed:  13.75691953401993  batches / s
batch:  4550    loss:  1.15215    speed:  13.446497574307754  batches / s
batch:  4560    loss:  1.11409    speed:  14.191648140306452  batches / s
batch:  4570    loss:  1.09937    speed:  14.3628531352668  batches / s
batch:  4580    loss:  1.12732    speed:  14.6855979738845  batches / s
batch:  4590    loss:  1.13363    speed:  14.32874768067219  batches / s
batch:  4600    loss:  1.14589    speed:  13.944219650556054  batches / s
batch:  4610    loss:  1.07469    speed:  12.989265586763182  batches / s
batch:  4620    loss:  1.1554    speed:  13.738

...

batch:  15500    loss:  0.844346    speed:  13.54335398066153  batches / s
batch:  15510    loss:  0.915989    speed:  13.625708008726969  batches / s
batch:  15520    loss:  0.896619    speed:  13.454554121480172  batches / s
batch:  15530    loss:  0.889219    speed:  13.847499893609122  batches / s
batch:  15540    loss:  0.896504    speed:  13.484145494893836  batches / s
batch:  15550    loss:  0.920318    speed:  13.366649592545915  batches / s
batch:  15560    loss:  0.874601    speed:  13.880304781987622  batches / s
batch:  15570    loss:  0.873082    speed:  14.285899670729451  batches / s
batch:  15580    loss:  0.860568    speed:  14.134067379905698  batches / s
batch:  15590    loss:  0.917554    speed:  13.650336657863912  batches / s
batch:  15600    loss:  0.881515    speed:  13.600116237870907  batches / s
batch:  15610    loss:  0.862079    speed:  13.013705876875747  batches / s
batch:  15620    loss:  0.87334    speed:  12.417856459687687  batches / s
batch:  15630 

batch:  16590    loss:  0.880385    speed:  13.411766978069545  batches / s
batch:  16600    loss:  0.89224    speed:  12.377138361596943  batches / s
batch:  16610    loss:  0.880937    speed:  12.829986883711634  batches / s
batch:  16620    loss:  0.892124    speed:  13.895714950301407  batches / s
batch:  16630    loss:  0.914965    speed:  13.624193879081018  batches / s
batch:  16640    loss:  0.863874    speed:  13.19427354666598  batches / s
batch:  16650    loss:  0.873561    speed:  13.48236535423392  batches / s
batch:  16660    loss:  0.869036    speed:  14.147070228974007  batches / s
batch:  16670    loss:  0.894707    speed:  12.579621674154907  batches / s
batch:  16680    loss:  0.873389    speed:  13.777101213689754  batches / s
batch:  16690    loss:  0.851762    speed:  13.046194364377873  batches / s
batch:  16700    loss:  0.852648    speed:  12.911563585072617  batches / s
batch:  16710    loss:  0.849316    speed:  12.53954948550615  batches / s
batch:  16720   

batch:  17680    loss:  0.90771    speed:  14.386916689085282  batches / s
batch:  17690    loss:  0.865828    speed:  13.198099850517336  batches / s
batch:  17700    loss:  0.875365    speed:  13.968758541464906  batches / s
batch:  17710    loss:  0.876805    speed:  13.663417664997347  batches / s
batch:  17720    loss:  0.880402    speed:  13.780436779271572  batches / s
batch:  17730    loss:  0.896511    speed:  13.517097534549247  batches / s
batch:  17740    loss:  0.884832    speed:  14.060251756809064  batches / s
batch:  17750    loss:  0.873253    speed:  13.205516722690017  batches / s
batch:  17760    loss:  0.893375    speed:  13.979805944093295  batches / s
batch:  17770    loss:  0.869144    speed:  14.184345452584619  batches / s
batch:  17780    loss:  0.86094    speed:  14.100086211374018  batches / s
batch:  17790    loss:  0.850895    speed:  14.119039653077857  batches / s
batch:  17800    loss:  0.888284    speed:  13.116319941209122  batches / s
batch:  17810 

batch:  18770    loss:  0.869546    speed:  13.018169552068997  batches / s
batch:  18780    loss:  0.841727    speed:  12.335285219855523  batches / s
batch:  18790    loss:  0.896316    speed:  13.807245012675311  batches / s
batch:  18800    loss:  0.921412    speed:  14.3557987353369  batches / s
batch:  18810    loss:  0.846896    speed:  14.351695134330718  batches / s
batch:  18820    loss:  0.853943    speed:  14.407715600353717  batches / s
batch:  18830    loss:  0.880274    speed:  14.06035639313836  batches / s
batch:  18840    loss:  0.845319    speed:  13.501540332332713  batches / s
batch:  18850    loss:  0.870728    speed:  13.228469931145348  batches / s
batch:  18860    loss:  0.861649    speed:  13.878959952903637  batches / s
batch:  18870    loss:  0.852794    speed:  13.906090562138992  batches / s
batch:  18880    loss:  0.886577    speed:  13.884788049539875  batches / s
batch:  18890    loss:  0.875655    speed:  13.99270050321697  batches / s
batch:  18900   

'./model.ckpt'

generate text

In [37]:
TEST_PREFIX = "прохожу и играюсь "

for i in range(len(TEST_PREFIX)):
    out = net.run_step( embed_to_vocab(TEST_PREFIX[i], vocab) , i==0)

print("SENTENCE:")
gen_str = TEST_PREFIX
for i in range(LEN_TEST_TEXT):
    element = np.random.choice( range(len(vocab)), p=out ) # Sample character from the network according to the generated output probabilities
    gen_str += vocab[element]

    out = net.run_step( embed_to_vocab(vocab[element], vocab) , False )
print(gen_str)

SENTENCE:
прохожу и играюсь несколько времени, одечной дымя через-плуту, и не окружают в себе не за пер-с; отчего косы ты пройду, чтобы быть послучше индцев после самым веселым и прочее, и сердце его собственного и прыснула. канкалыва желтые, ни за что не хотел вы, будеть, всё равно. он относит к петца и пошли и кого ободрилась!.. да и зачем же, дмитрий прокофьич? слушал он вполне. разве это возможно? слышите! фру, он жениться родиом родятся, — продолжал разумихин, — но не стыдно со всеми экономи средства и в самых двадцат
